<a href="https://colab.research.google.com/github/Naveen1023/Machine-Learning-Experiments-on-Ciphers/blob/main/Trivium_Round_1_DRDO3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

import tensorflow as tf

In [ ]:
bits = []


In [ ]:
from collections import deque
from itertools import repeat
from sys import version_info


class Trivium:
    
    def __init__(self, key, iv):
        """in the beginning we need to transform the key as well as the IV.
        Afterwards we initialize the state."""
        self.state = None
        self.counter = 0
        self.key = key  # self._setLength(key)
        self.iv = iv  # self._setLength(iv)

        # Initialize state
        # len 100
        init_list = list(map(int, list(self.key)))
        init_list += list(repeat(0, 20))
        # len 84
        init_list += list(map(int, list(self.iv)))
        init_list += list(repeat(0, 4))
        # len 111
        init_list += list(repeat(0, 108))
        init_list += list([1, 1, 1])
        self.state = deque(init_list)

        # Do 4 full cycles, drop output
        for i in range(1):
            self._gen_keystream()

    def encrypt(self, message):
        """To be implemented"""
        pass

    def decrypt(self, cipher):
        """To be implemented"""
        pass

    def keystream(self):
        """output keystream
        only use this when you know what you are doing!!"""
        while self.counter < 2**64:
            self.counter += 1
            yield self._gen_keystream()

    def _setLength(self, input_data):
        """we cut off after 80 bits, alternatively we pad these with zeros."""
        input_data = "{0:080b}".format(input_data)
        if len(input_data) > 80:
            input_data = input_data[:(len(input_data)-81):-1]
        else:
            input_data = input_data[::-1]
        return input_data

    def _gen_keystream(self):
        """this method generates triviums keystream"""

        a_1 = self.state[90] & self.state[91]
        a_2 = self.state[181] & self.state[182]
        a_3 = self.state[292] & self.state[293]

        t_1 = self.state[65] ^ self.state[92]
        t_2 = self.state[168] ^ self.state[183]
        t_3 = self.state[249] ^ self.state[294]

        out = t_1 ^ t_2 ^ t_3

        s_1 = a_1 ^ self.state[177] ^ t_1
        s_2 = a_2 ^ self.state[270] ^ t_2
        s_3 = a_3 ^ self.state[68] ^ t_3

        self.state.rotate(1)

        self.state[0] = s_3
        self.state[100] = s_1
        self.state[184] = s_2

        return out


def main():
    keystream = []
    KEY = hex_to_bits("0B62B5085BAE0154A7FA")[::-1]
    IV = hex_to_bits("288FF65DC42B92F960C7")[::-1]
    trivium = Trivium(KEY, IV)

    # Check python version
    if version_info[0] == 2:
        next_key_bit = trivium.keystream().next
    elif version_info[0] == 3:
        next_key_bit = trivium.keystream().__next__
    else:
        print("invalid python version")
        return

    for i in range(1):
        
        for j in range(25700000):
            keystream.append(next_key_bit())
            bits.append(next_key_bit())
            
#         print(keystream)
    

# Convert strings of hex to strings of bytes and back, little-endian style
_allbytes = dict([("%02X" % i, i) for i in range(256)])


def _hex_to_bytes(s):
    return [_allbytes[s[i:i+2].upper()] for i in range(0, len(s), 2)]


def hex_to_bits(s):
    return [(b >> i) & 1 for b in _hex_to_bytes(s)
            for i in range(8)]

def bits_to_hex(b):
    return "".join(["%02X" % sum([b[i + j] << j for j in range(8)])
                    for i in range(0, len(b), 8)])


if __name__ == "__main__":
    main()

In [ ]:
matrix = np.empty((10000,2560))

In [ ]:
data1D = []

for i in range(0,25696991):
    temp_str = ""
    for k in range(i, i+8):
        temp_str += str(bits[k])

    decimal = int(temp_str, 2)
    data1D.append(decimal)
      

In [ ]:
print("length of decimat data 1D - ", len(data1D))
print(data1D[0:3])

length of decimat data 1D -  25696991
[8, 17, 34]


In [ ]:
# add elements to fratures matrix of 10000*2560

k = 0
for i in range(0,10000):
    for j in range(0,2560):
        matrix[i][j] = data1D[k]
        k+=1
# print(matrix)
print("shape of output matrix-",matrix.shape)

shape of output matrix- (10000, 2560)


In [ ]:
# create labels now 
labels =[]
for i in range(2568,25700000,2560):
    labels.append(bits[i])
print(len(labels ))

10039


In [ ]:
labels = labels[0:10000]
print(len(labels))


10000


In [ ]:
labels = np.reshape(labels, 10000, 1)
print(labels.shape)

(10000,)


In [ ]:
X = matrix
Y = labels

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [ ]:
print(len(x_train))

8000


In [ ]:
# normalize the data 
# x_train = tf.keras.utils.normalize(x_train, axis = 1)
# x_test = tf.keras.utils.normalize(x_test, axis = 1)
x_train=x_train/255
x_test=x_test/255


x_train=x_train.reshape(len(x_train),1,2560)
x_test=x_test.reshape(len(x_test),1,2560)


[[0.30980392 0.61960784 0.23529412 ... 0.79607843 0.58823529 0.17254902]]


In [ ]:
import keras 
from keras.models import Sequential 
from keras.layers import Dense, Flatten
from keras.layers import Embedding
from keras.layers import LSTM


Using TensorFlow backend.


In [ ]:
model = Sequential()

model.add(LSTM(100, input_shape = (1,2560),activation='relu')) 
# model.add(LSTM(200, input_shape = (None,2560),activation='relu')) 
model.add(Dense(150, activation = 'relu'))
# model.add(Flatten())
model.add(Dense(output_dim=1, activation='tanh'))
print('Model loaded.')
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=['accuracy'])
# print(model.summary())

Model loaded.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", units=1)`
  import sys


In [ ]:

for i in range(0,8000,10):
  x_train_instance = x_train[i:i+10]
  y_train_instance = y_train[i:i+10]
  x_train_instance = x_train_instance.reshape(len(x_train_instance),1,2560)
  model.fit(x_train_instance, y_train_instance, epochs = 1)


In [ ]:
x_test = x_test.reshape(len(x_test),1,2560)

print(len(x_test))


2000


In [ ]:
y_pred = model.predict(x_test)

val_loss , val_acc = model.evaluate(x_test,y_test)

print(val_loss, val_acc)

2000/2000 [==============================] - 0s 110us/step
0.6944691257476807 0.5165


In [ ]:
for i in range(0,100):
  print(y_pred[i],"  ",y_test[i])

[0.48937514]    1
[0.48670194]    0
[0.46687955]    0
[0.5068428]    0
[0.47497302]    1
[0.46555966]    1
[0.46880883]    1
[0.4762402]    0
[0.45186496]    0
[0.5316354]    0
[0.47416973]    0
[0.4823043]    0
[0.5366808]    1
[0.4782168]    1
[0.49474987]    1
[0.46192425]    0
[0.5489923]    0
[0.4400134]    0
[0.46181476]    1
[0.46634793]    0
[0.49841222]    1
[0.47819722]    1
[0.470451]    0
[0.47630757]    0
[0.4644366]    0
[0.46948886]    1
[0.4823169]    1
[0.46863753]    1
[0.4890213]    1
[0.48537284]    1
[0.4430279]    0
[0.5017867]    0
[0.51045716]    1
[0.50509655]    1
[0.48969042]    1
[0.4898807]    1
[0.4914319]    1
[0.5197429]    1
[0.53205264]    1
[0.5013547]    0
[0.49345586]    1
[0.47034898]    1
[0.44467974]    0
[0.4654405]    1
[0.4626807]    1
[0.47112608]    1
[0.49208832]    1
[0.50456]    0
[0.48403162]    1
[0.48622268]    0
[0.51018834]    0
[0.49143013]    1
[0.5422392]    0
[0.47975144]    1
[0.48453262]    1
[0.5047948]    0
[0.50755537]    1


In [ ]:
y_pred = (y_pred > 0.5)
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

print(cm)

[[647 334]
 [633 386]]
